In [1]:
import numpy as np
import matplotlib.pyplot as plt
from shapely import Polygon,Point
from scipy.spatial import ConvexHull
from itertools import combinations
from shapely.ops import unary_union

In [97]:
imp = .5
X = np.array([(-3,-1), (-2, 3),(0.5 ,-4),(1.5,6),(3.2,2)])
Bx= np.zeros((len(X),4,2));
for i in range(len(X)):
    Bx[i][0][0] = X[i][0]-imp;
    Bx[i][0][1] = X[i][1]+imp;
    Bx[i][1][0] = X[i][0]+imp;
    Bx[i][1][1] = X[i][1]+imp;
    Bx[i][2][0] = X[i][0]+imp;
    Bx[i][2][1] = X[i][1]-imp;
    Bx[i][3][0] = X[i][0]-imp;
    Bx[i][3][1] = X[i][1]-imp;



In [77]:
print(X[1,0])

-2.0


In [80]:
def getPotRegions(X):
    Bx= np.zeros((len(X),4,2));
    for i in range(len(X)):
        Bx[i,0,:]= [X[i][0]-imp, X[i][1]+imp]
        Bx[i,1,:]= [X[i][0]+imp, X[i][1]+imp]
        Bx[i,2,:]= [X[i][0]+imp, X[i][1]-imp]
        Bx[i,3,:]= [X[i][0]-imp, X[i][1]-imp]
    return(Bx);

In [4]:
def plothull(X):
    hull = ConvexHull(X);
    hullx = np.append(X[hull.vertices,0],X[hull.vertices[0],0]);
    hully = np.append(X[hull.vertices,1],X[hull.vertices[0],1]);
    plt.plot(hullx,hully);
    

In [5]:

comb3 = list(combinations(np.arange(len(X)),3))
safe = [];
dp1 =[]
for i in range(len(comb3)):
    comb2 = list(combinations(comb3[i][:],2))
    b1 = int(comb3[i][0]);
    b2 = int(comb3[i][1]);
    b3 = int(comb3[i][2]);
    dp1 = np.vstack((Bx[b1,:,:],Bx[b2,:,:],Bx[b3,:,:]));
    idx = ConvexHull(dp1);
    dp1 = Polygon(dp1[idx.vertices,:]);
    dp2 = Polygon();
    for j in range(len(comb2)):
        a1 = comb2[j][0];
        a2 = comb2[j][1];
        temp= np.vstack((Bx[a1,:,:],Bx[a2,:,:]));
        idx = ConvexHull(temp);
        temp =  Polygon(temp[idx.vertices,:]);
        dp2 = unary_union([dp2,temp])
        diff = dp1.difference(dp2);
        x,y = np.array(diff.exterior.xy)
        safe.append([x,y])

In [51]:
def calcsafepoint(n,Bx):
    n = np.arange(n);
    comb3 = list(combinations(n,3));
    safeX = [];
    safeY =[];
    dp1 =[]
    for i in range(len(comb3)):
        comb2 = list(combinations(comb3[i][:],2))
        b1 = int(comb3[i][0]);
        b2 = int(comb3[i][1]);
        b3 = int(comb3[i][2]);
        dp1 = np.vstack((Bx[b1,:,:],Bx[b2,:,:],Bx[b3,:,:]));
        idx = ConvexHull(dp1);
        dp1 = Polygon(dp1[idx.vertices,:]);
        dp2 = Polygon();
        for j in range(len(comb2)):
            a1 = comb2[j][0];
            a2 = comb2[j][1];
            temp= np.vstack((Bx[a1,:,:],Bx[a2,:,:]));
            idx = ConvexHull(temp);
            temp =  Polygon(temp[idx.vertices,:]);
            dp2 = unary_union([dp2,temp])
        diff = dp1.difference(dp2);
        x,y = np.array(diff.exterior.xy)
        safeX.append(x);
        safeY.append(y);
    return [safeX,safeY];

In [52]:
n = len(X);
safeX,safeY = calcsafepoint(n,Bx)

In [75]:
def meansafepoint(safeX,safeY):
    mx =0;
    my = 0;
    count = 0;
    for i in range(len(safeX)):
        if len(safeX[i] != 0):
            mx += np.mean(safeX[i]);
            my += np.mean(safeY[i]);
            count+=1;
    mx = mx/count;
    my = my/count;
    return mx,my;



0


In [91]:
def plotagents(Bx,safeX,safeY,fig,ax):
    fig.clear()
    for i in range(len(safeX)):
        x = safeX[i][:];
        y = safeY[i][:];
        ax.fill(x,y);
    for j in range(n):
        x = Bx[j,:,0];
        y = Bx[j,:,1];
        ax.scatter(x,y);
    fig.canvas.draw()
    fig.canvas.flush_events();
    plt.show();

In [98]:
fig,ax = plt.subplots(figsize=(4,5));
plt.ion();
for t in range(200):
    dx = .01;
    n = len(X);
    target = np.zeros((n,2));
    for i in range(n):
        Bx = getPotRegions(X);
        safeX,safeY = calcsafepoint(n,Bx);
        target[i] = meansafepoint(safeX,safeY);
        
    for i in range(n):
        X[i]= X[i]+dx*(target[i]-X[i]);
    plotagents(Bx,safeX,safeY,fig,ax);
    
        

<Figure size 400x500 with 0 Axes>

ZeroDivisionError: division by zero